In [1]:
# This notebook contains steps to create a sparse matrix of features from the training dataset
# The id, device_id, device_ip and click columns are left out
# Object columns are cast as categorical and one-hot-encoded, then joined to a sparse CSR matrix of numeric features
# The resulting sparse matrix (~29k features x 40MM rows) has been stored as an .npz file in the /assets/ directory
# Clicks from the training set have been stored in a CSV file called y_train
# TODO (if needed) - check to make sure that correct index values have been persisted through transformations and joining

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from scipy import sparse


### Test Data and Sample Submission

Loading in our full test file as well as the sample submission CSV

In [3]:
# loading in test data as well as the Sample Submission file
sample = pd.read_csv('../assets/sampleSubmission')
test = pd.read_csv('../assets/test')

### Training Data

For this notebook we are only loading 20% of our full training dataset. This will allow us to experiment with feature engineering on a local device with 8GB of memory. Running the same operations against the full set may require a different solution (eg AWS Sagemaker or Databricks Spark Cluster).

In [4]:
# Because of GitHub space limits (no files over 2GB), train data file was split into 5 pieces

# Loading the first file with header row to use for column names
%time train = pd.read_csv("../assets/trainaa")
print("trainaa loaded")

# Loading subsequent files as separate dataframes with common column names
#trainab = pd.read_csv("../assets/trainab", header=None, names=trainaa.columns)
#print("trainab loaded")
#trainac = pd.read_csv("../assets/trainac", header=None, names=trainaa.columns)
#print("trainac loaded")
#trainad = pd.read_csv("../assets/trainad", header=None, names=trainaa.columns)
#print("trainad loaded")
#trainae = pd.read_csv("../assets/trainae", header=None, names=trainaa.columns)
#print("trainae loaded")

# Concatenating all files into one training set for EDA purposes
#%time train = pd.concat([trainaa, trainab, trainac, trainad, trainae], ignore_index=True)
train.head(1)

trainaa loaded


In [5]:
train = train.to_sparse()

In [6]:
# creating a list of categorical columns
# dropping device_id and device_ip columns (with 2MM+ and 6MM+ values) for now
obj_cols_train = train.select_dtypes(include='object').drop(columns=['device_id',  'device_ip'])
obj_cols_test = test.select_dtypes(include='object').drop(columns=['device_id',  'device_ip'])

In [7]:
obj_cols_train.dtypes

site_id          object
site_domain      object
site_category    object
app_id           object
app_domain       object
app_category     object
device_model     object
dtype: object

In [8]:
obj_cols_test.dtypes

site_id          object
site_domain      object
site_category    object
app_id           object
app_domain       object
app_category     object
device_model     object
dtype: object

In [9]:
# trying one colum at a time
%time obj_dummies_train_1 = pd.get_dummies(train[['site_id', 'site_domain', 'site_category']], sparse=True)

KeyboardInterrupt: 

In [ ]:
%time obj_dummies_train_2 = pd.get_dummies(train[['app_id', 'app_domain', 'app_category', 'device_model']])

In [ ]:
obj_dummies_test = pd.get_dummies(obj_cols_test)

In [ ]:
train_dummies = obj_dummies_train_1.merge(obj_dummies_train_2, left_index=True, right_index=True)
train_dummies.head()

In [ ]:
#%train dummies_aligned_train, dummies_aligned_test = obj_dummies_train.align(obj_dummies_test, fill_value=0)

In [ ]:
# Label Encoding the object columns to make them into 1s and 0s

#le_train = LabelEncoder()
#le_test = LabelEncoder()

#%time obj_cols_enc_train = obj_cols_train.apply(le_train.fit_transform)
#obj_cols_enc_test = obj_cols_test.apply(le_test.fit_transform)

In [ ]:
assert red = blue

In [ ]:
# setting columns as categorical so we can generate sparse matrix
obj_cols_train = obj_cols_train.apply(lambda x: x.astype('category'))
obj_cols_test = obj_cols_test.apply(lambda x: x.astype('category'))


In [ ]:
# checking shapes
print(f"Train object cols encoded shape: {obj_cols_enc_train.shape}\n Test object cols encoded shape: {obj_cols_enc_test.shape}")




In [ ]:
# One Hot Encoding the categorical columns

ohe_train = OneHotEncoder()
ohe_test = OneHotEncoder()

%time onehot_train = ohe_train.fit_transform(obj_cols_enc_train)
onehot_test = ohe_test.fit_transform(obj_cols_enc_test)

In [ ]:
object_cols_onehot.shape

In [ ]:
# OneHot returned a sparse matrix

type(object_cols_onehot)

In [ ]:
# Y (which we will need for validation)
y = train.click.copy()

# Test IDs (which we will need for sample submission)
test_ids = test['id'].copy()

In [ ]:
# Numeric columns
num_cols_train = train.select_dtypes(exclude='object').drop(columns=['id','click'])

num_cols_test = test.select_dtypes(exclude='object').drop(columns=['id'])

In [ ]:
# Normalizing numeric columns 

ss_train = StandardScaler()
ss_test = StandardScaler()

%time num_cols_scaled_train = ss_train.fit_transform(num_cols_train)
num_cols_scaled_test = ss_test.fit_transform(num_cols_test)


In [ ]:
num_cols_train_scaled.shape

In [ ]:
num_cols_test_scaled.shape

In [ ]:
# Making a sparse matrix from the scaled numeric columns

#%time num_sparse_train = sparse.csr_matrix(num_cols_scaled_train)
#num_sparse_test = sparse.csr_matrix(num_cols_scaled_test)

In [ ]:
# checking shape

#numeric_sparse.shape#

In [ ]:
# assembling a complete sparse matrix from numeric and dummy features

#%time sparse_features = sparse.hstack([numeric_sparse, object_cols_onehot])

In [ ]:
# checking shape - ~1 Trillion "cells" represented
#sparse_features.shape

In [ ]:
# save sparse matrix to a file

#%time saved = sparse.save_npz('../assets/sparse_no_dev_id_dev_ip', sparse_features)

In [ ]:
# save y to a file

#%time y_saved = y.to_csv('../assets/y_train')